|        |        |        |
|--------|--------|--------|
![H-BRS](logos/h-brs.png) | ![A2S](logos/a2s.png) | ![b-it](logos/b-it.png) |

# Autonomous Mobile Robots

# AMR Assignment 5

### General information

* Please do not add or delete any cells. Answers belong into the already provided cells (below the question).
* If a function is given (either as a signature or a full function), you should not change the name, arguments, or return value of the function.
* If you encounter empty cells underneath the answer that can not be edited, please ignore them; they are for testing purposes.
* Please note that variables declared in the notebook cells have global scope. To make sure your assignment works correctly as a whole, please restart the kernel and run all cells before submitting (e.g. via *Kernel -> Restart & Run All*).
* Code cells where you are supposed to give your answer often include the line  ```raise NotImplementedError```. This makes it easier to automatically grade answers. Once you fill out a function, please delete this line.

### Submission

Please make sure to write all your team members 2s IDs in the cell below before submission. Please submit your notebook via the JupyterHub web interface (in the main view -> Assignments -> Submit). If it is a group assignment, please make only one submission per group (for easier bookkeeping, it is best if this is always the same team member).

### Questions about the assignment

If you have questions about the assignment, you are encouraged to post them in the LEA forum. Proactive discussions lead to better understanding. Let's keep the forum active.

## Team members (2s IDs):

YOUR ANSWER HERE

* Mmemon2s
*
*

# Landmark-based Localisation using Kalman Filter (100 points)

In this assignment, we will use a Kalman filter for localisation in an environment based on the measurement of landmark locations. As you already know, sensor readings are affected by noise, making it difficult to achieve accurate measurement of the physical quantity under consideration. The Kalman filter, which belongs to the family of Bayes filter algorithms, can be used for minimising the error in the estimation. The belief of the state (in this case, it is the **pose of the robot** with respect to the **map frame**) is represented as a Gaussian distribution. The objective is to estimate the mean $\mu$ and covariance matrix $\Sigma$ for the physical quantity under consideration at every time instance.

In this assignment, the Robile is placed in a simulation environment in Gazebo, where several RFID tags are positioned on the ground. Now, suppose that a virtual RFID detector is mounted on the robot, sharing the *same frame* and *field of view* as the LiDAR. In the real world, the robot's real pose almost never matches with its ideal motion model due to different sources of noise, such as from the encoder readings, unstructured environment interaction, as well as other noisy perception data.

In this assignment, we will explore how the Kalman filter can be employed to improve the estimation of the robot's pose in this setup. Below is the simulation environment where the RFID tags are highlighted.

![robile_localisation_kalman](img/robile_localisation_kalman.png)

The positions of the RFID tags as $(x,y)$ coordinates in the `odom` frame with tag names are as follows:  
- A: $(1.0, 1.0)$
- B: $(6.0, 1.0)$
- C: $(3.0, -1.0)$
- D: $(1.0, -3.0)$
- E: $(4.0, -4.0)$

### Motion noise consideration

When the robot is moving in the environment, the noise in the data published onto the `/odom` topic is negligible in the simulation. To simulate the real-world noise, for every message on the `/odom` topic, some noise is introduced manually in the associated *robile_rfid_tag_finder.py* script, and the variable `real_base_link_pose` is updated accordingly. Here, the data on the `/odom` topic, and consequently the visualization of the robot model in RVIZ, can be considered as an ideal motion model, which is the motion estimate without noise; the data on `real_base_link_pose` is then the actual pose of the robot's base (`real_base_link` frame) with respect to which the RFID tags are measured, and which are available on the `/rfid_tag_poses` topic. The `real_base_link_pose` topic thus represents the actual pose of the robot, which deviates from `/odom` due to uncertain interaction with the environment.

Your goal is to read from the `odom` topic and, along with the measured RFID tag positions, estimate the posterior belief of the robot's pose and the variance associated with it. This estimated pose, which should be published to the `estimated_base_link_pose` topic, should eventually converge to the pose represented on `real_laser_link_pose`.

### Summary

- The pose of the robot with respect to the odom frame, published on the `/odom` topic represents ideal motion without noise.
- Since it is challenging to mathematically represent or model all physical interactions with the environment, in reality, the robot deviates from the ideal motion and its pose estimation is affected by noise in the sensor data and an incomplete model of the interaction with the environment. The actual robot pose is considered to be the `real_base_link` frame.
- The measurement model is the pose estimate using the detected RFID tags on the `/rfid_tag_poses` topic, which is represented by a custom message of type ``PoseLabelledArray``. These tags are measured with respect to the `real_base_link` frame.
- Note that the `real_base_link` frame is only used to visualise the actual robot pose; thus, only the data on `/odom` and `/rfid_tag_poses` topics should be used to determine robot's posterior pose `estimated_base_link_pose`.


### Notes

- The initial `real_base_link` frame is considered to be overlapping with the `odom` frame.
- To determine the robot's pose by the measurement of RFID tag positions, at least two tags needs to be perceived. The problem could be made simpler by considering the pose estimated using these RFID positions as measurement data rather than the measured RFID positions themselves.
- The initial state, i.e., the `estimated_base_link_pose` pose at the start of simulation is considered the same as the pose of the `odom` frame, namely `estimated_base_link_pose` can be considered as initially known with zero or constant uncertainty. Optionally, it can be updated using the `2D Pose Estimate` widget in RVIZ, in which case the mean and variance of the estimate should be reset.

### Instructions

Upload the modified `robile_navigation` package with your submission, and also paste your Kalman filter implementation in the cell below. Before starting your implementation please follow these steps:

- Modify the simulation launch file to load the `closed_walls.world` environment from the `robile_gazebo` package
- Implement pose estimation using a Kalman filter in the `robile_localisation_kalman.py` script from the `robile_navigation` repository
- Use the launch file `landmark_based_localisation.launch.py` under `robile_navigation` and modify it accordingly to launch your implementation
- Please feel free to modify any of the scripts according to your needs

### Potentially useful references

- Section 5.6.8.5 in the "Introduction to Autonomous Mobile Robots" book. It doesn't have the same setup, but is still useful
- https://www.alanzucconi.com/2022/07/24/kalman-gain/
- http://bilgin.esme.org/BitsAndBytes/KalmanFilterforDummies

In [ ]:
### Paste your code from robile_localisation_kalman.py in this cell

# YOUR CODE HERE

#!/usr/bin/env python3

import rclpy
from rclpy.node import Node
from geometry_msgs.msg import PoseStamped
from robile_interfaces.msg import PositionLabelledArray
from nav_msgs.msg import Odometry
import numpy as np
import tf2_ros
from tf_transformations import euler_from_quaternion, quaternion_from_euler
import time

class LocalisationUsingKalmanFilter(Node):
    """
    Landmark based localisation using Kalman Filter
    This is a partially structured class for AMR assignment
    """

    def __init__(self):
        super().__init__('localisation_using_kalman_filter')

        # declaring and getting parameters from yaml file
        self.declare_parameters(
            namespace='',
            parameters=[
                ('map_frame', 'map'),
                ('odom_frame', 'odom'),                
                ('laser_link_frame', 'base_laser_front_link'),
                ('real_base_link_frame', 'real_base_link'),
                ('scan_topic', 'scan'),
                ('odom_topic', 'odom'),
                ('rfid_tag_poses_topic', 'rfid_tag_poses'),
                ('initial_pose_topic', 'initialpose'),
                ('real_base_link_pose_topic', 'real_base_link_pose'),
                ('estimated_base_link_pose_topic', 'estimated_base_link_pose'),
                ('minimum_travel_distance', 0.1),
                ('minimum_travel_heading', 0.1),
                ('rfid_tags.A', [0.,0.]),
                ('rfid_tags.B', [0.,0.]),
                ('rfid_tags.C', [0.,0.]),
                ('rfid_tags.D', [0.,0.]),
                ('rfid_tags.E', [0.,0.]),                        
            ])

        self.map_frame = self.get_parameter('map_frame').get_parameter_value().string_value
        self.odom_frame = self.get_parameter('odom_frame').get_parameter_value().string_value
        self.laser_link_frame = self.get_parameter('laser_link_frame').get_parameter_value().string_value
        self.real_base_link_frame = self.get_parameter('real_base_link_frame').get_parameter_value().string_value
        self.scan_topic = self.get_parameter('scan_topic').get_parameter_value().string_value
        self.odom_topic = self.get_parameter('odom_topic').get_parameter_value().string_value
        self.rfid_tag_poses_topic = self.get_parameter('rfid_tag_poses_topic').get_parameter_value().string_value
        self.initial_pose_topic = self.get_parameter('initial_pose_topic').get_parameter_value().string_value
        self.real_base_link_pose_topic = self.get_parameter('real_base_link_pose_topic').get_parameter_value().string_value
        self.estimated_base_link_pose_topic = self.get_parameter('estimated_base_link_pose_topic').get_parameter_value().string_value
        self.minimum_travel_distance = self.get_parameter('minimum_travel_distance').get_parameter_value().double_value
        self.minimum_travel_heading = self.get_parameter('minimum_travel_heading').get_parameter_value().double_value
        self.rfid_tags_A = self.get_parameter('rfid_tags.A').get_parameter_value().double_array_value
        self.rfid_tags_B = self.get_parameter('rfid_tags.B').get_parameter_value().double_array_value
        self.rfid_tags_C = self.get_parameter('rfid_tags.C').get_parameter_value().double_array_value
        self.rfid_tags_D = self.get_parameter('rfid_tags.D').get_parameter_value().double_array_value
        self.rfid_tags_E = self.get_parameter('rfid_tags.E').get_parameter_value().double_array_value
        self.lastOdometry = None

        # setting up laser scan and rfid tag subscribers
        self.rfid_tag_subscriber = self.create_subscription(PositionLabelledArray, self.rfid_tag_poses_topic, self.rfid_callback, 10)
        self.real_laser_link_subscriber = self.create_subscription(PoseStamped, self.real_base_link_pose_topic, self.real_base_link_pose_callback, 10)        
        self.odom_subscriber = self.create_subscription(Odometry, self.odom_topic, self.updateBelief, 10)
        self.estimated_robot_pose_publisher = self.create_publisher(PoseStamped, self.estimated_base_link_pose_topic, 10)
        
        # setting up tf2 listener
        self.tf_buffer = tf2_ros.Buffer()
        self.tf_listener = tf2_ros.TransformListener(self.tf_buffer, self)

        self.state = np.array([0.,0.,0.]) # This is x0 which contains x,y,theta
        self.cov = np.eye(3,3)*1e-8
        self.mutexUpdate = False

        # In homogeneous coordinates
        self.odomRFID = {
            "A": np.array([1.0, 1.0, 1.0]),
            "B": np.array([6.0, 1.0, 1.0]),
            "C": np.array([3.0, -1.0, 1.0]),
            "D": np.array([1.0, -3.0, 1.0]),
            "E": np.array([4.0, -4.0, 1.0])
        }

        self.tf_buffer = tf2_ros.Buffer()
        self.tf_listener = tf2_ros.TransformListener(self.tf_buffer, self)

    def getTransformation(self):
        x,y,theta = self.state
        s = np.sin(theta)
        c = np.cos(theta)
        return np.array([
            [c,-s,x],
            [s,c,y],
            [0,0,1],
        ])
    
    def quatToAngle(self, quat):
        return euler_from_quaternion([quat.x, quat.y, quat.z, quat.w])[2]
    
    def extractSubCov(self, cov):
        return cov.reshape(6,6)[(0,1,5), :][:, (0,1,5)]
    
    def getBeliefRFIDError(self, detectedRFID):
        tranformation = self.getTransformation()

        idToIdx = {"A":0,"B":1,"C":2,"D":3,"E":4}
        beliefRFIDError = np.zeros((6,1))
        for el in detectedRFID.positions:
            idx = idToIdx[el.name]
            transformedPoint = np.dot(tranformation,self.odomRFID[el.name].T).T
            #6x1
            beliefRFIDError[idx] = np.sqrt((el.position.x - transformedPoint[0])**2 + (el.position.y - transformedPoint[1])**2)

        return beliefRFIDError

    def computeKalmanGain(self):
        H = np.ones((6,3))
        R = np.eye(6,6) * 0.01
        t1 = np.matmul(self.cov, H.T)
        t2 = np.matmul(H,np.matmul(self.cov, H.T))+R
        K = np.matmul(t1, np.linalg.inv(t2))
        return K 

    def rfid_callback(self, msg: PositionLabelledArray):
        """
        Based on the detected RFID tags, performing measurement update
        """        
        if not self.mutexUpdate:
            return
        
        self.mutexUpdate = False
        poseEstimation = PoseStamped()

        K = self.computeKalmanGain()

        be = self.getBeliefRFIDError(msg)

        self.state += np.dot(K,be)[:,0]

        self.cov = np.dot((np.eye(3,3) - np.dot(K, np.ones((6,3)))), self.cov)

        poseEstimation.header.stamp = self.get_clock().now().to_msg()
        poseEstimation.header.frame_id = self.map_frame
        poseEstimation.pose.position.x = self.state[0]
        poseEstimation.pose.position.y = self.state[1]
        poseEstimation.pose.position.z = 0.
        quat = quaternion_from_euler(0,0,self.state[2])
        poseEstimation.pose.orientation.x = quat[0]
        poseEstimation.pose.orientation.y = quat[1]
        poseEstimation.pose.orientation.z = quat[2]
        poseEstimation.pose.orientation.w = quat[3]

        print("LINK POSE REAL", self.real_laser_link_pose)
        print("STATE", self.state)

        self.estimated_robot_pose_publisher.publish(poseEstimation)

    def real_base_link_pose_callback(self, msg):
        """
        Updating the base_link pose based on the update in robile_rfid_tag_finder.py
        """
        yaw = euler_from_quaternion([msg.pose.orientation.x, msg.pose.orientation.y, msg.pose.orientation.z, msg.pose.orientation.w])[2]
        self.real_laser_link_pose = [msg.pose.position.x, msg.pose.position.y, yaw]

    def updateBelief(self, msg:Odometry):
        if not self.lastOdometry:
            self.lastOdometry = msg
            return 

        linearOfft = np.array([msg.pose.pose.position.x - self.lastOdometry.pose.pose.position.x, 
                               msg.pose.pose.position.y - self.lastOdometry.pose.pose.position.y ])
        angularOfft = self.quatToAngle(msg.pose.pose.orientation) - self.quatToAngle(self.lastOdometry.pose.pose.orientation)

        if np.linalg.norm(linearOfft) < self.minimum_travel_distance and abs(angularOfft) < self.minimum_travel_heading:
            return
        
        covOfft = self.extractSubCov(msg.pose.covariance) - self.extractSubCov(self.lastOdometry.pose.covariance)

        self.state += np.append(linearOfft, angularOfft)
        self.cov += covOfft
        self.lastOdometry = msg

        self.mutexUpdate = True


def main(args=None):
    rclpy.init(args=args)

    try:
        localisation_using_kalman_filter = LocalisationUsingKalmanFilter()
        rclpy.spin(localisation_using_kalman_filter)

    finally:
        localisation_using_kalman_filter.destroy_node()
        rclpy.shutdown()

if __name__ == '__main__':
    main()
# raise NotImplementedError()

Finally, comment on your observations while evaluating your implementation in the cell below, and include relevant screenshots to verify your implementation.

YOUR ANSWER HERE